In [1]:
# IMPORT NECESSARY LIBRARIES
from tensorflow.keras.models import load_model
import librosa
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
from IPython.display import Audio
import numpy as np
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
from sklearn.metrics import confusion_matrix
import IPython.display as ipd  # To play sound in the notebook
import os # interface with underlying OS that python is running on
import sys
import warnings
# ignore warnings 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from tensorflow.keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization, Dense
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.12.2 when it was built against 1.12.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


Init Plugin
Init Graph Optimizer
Init Kernel


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
model = load_model("models/Emotion_Model.h5")

Metal device set to: Apple M1 Pro


2022-05-06 02:00:35.287119: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-06 02:00:35.287233: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
pred_wav_for_test = "datasets/augmented/upset/data_noise01_padding_re-cry- (39).wav"


In [4]:
df = pd.DataFrame(columns=['features'])


In [5]:
def get_features(path):

    X, sample_rate = librosa.load(path, res_type='kaiser_fast', duration=10.0, sr=None)
    
    stft = np.abs(librosa.stft(X))

    # fmin 和 fmax 對應於人類語音的最小最大基本頻率
    pitches, magnitudes = librosa.piptrack(X, sr=sample_rate, S=stft, fmin=70, fmax=400)
    pitch = []
    for i in range(magnitudes.shape[1]):
        index = magnitudes[:, 1].argmax()
        pitch.append(pitches[index, i])

    pitch_tuning_offset = librosa.pitch_tuning(pitches)
    pitchmean = np.mean(pitch)
    pitchstd = np.std(pitch)
    pitchmax = np.max(pitch)
    pitchmin = np.min(pitch)

    # 頻譜質心
    cent = librosa.feature.spectral_centroid(y=X, sr=sample_rate)
    cent = cent / np.sum(cent)
    meancent = np.mean(cent)
    stdcent = np.std(cent)
    maxcent = np.max(cent)

    # 譜平面
    flatness = np.mean(librosa.feature.spectral_flatness(y=X))

    # 使用系數為13的MFCC特徵
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13).T, axis=0)
    mfccsstd = np.std(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13).T, axis=0)
    mfccmax = np.max(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13).T, axis=0)

    # 色譜圖
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)

    # 梅爾頻率
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)

    # ottava對比
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)

    # 過零率
    zerocr = np.mean(librosa.feature.zero_crossing_rate(X))

    S, phase = librosa.magphase(stft)
    meanMagnitude = np.mean(S)
    stdMagnitude = np.std(S)
    maxMagnitude = np.max(S)

    # 均方根能量
    rms = librosa.feature.rms(S=S)[0]
    meanrms = np.mean(rms)
    stdrms = np.std(rms)
    maxrms = np.max(rms)

    ext_features = np.array([
        flatness, zerocr, meanMagnitude, maxMagnitude, meancent, stdcent,
        maxcent, stdMagnitude, pitchmean, pitchmax, pitchstd,
        pitch_tuning_offset, meanrms, maxrms, stdrms
    ])

    ext_features = np.concatenate((ext_features, mfccs, mfccsstd, mfccmax, chroma, mel, contrast))
        
    return ext_features

In [6]:
df.features = get_features(pred_wav_for_test)
print(len(df))
df.head()

201


,features
0,0.103338
1,0.178215
2,0.880074
3,163.583435
4,0.003195


In [7]:
textfeature_df = pd.DataFrame(df['features'].values.tolist()).T
textfeature_df

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,0.103338,0.178215,0.880074,163.583435,0.003195,0.000966,0.005456,3.649114,37.56646,339.463165,...,0.010984,0.009696,0.009947,15.474105,19.077137,22.653144,20.953209,18.644379,15.410624,13.985384


In [8]:
textfeature_df.to_csv('testfeatures.csv', index=False)

In [9]:
textfeature_df = pd.read_csv('testfeatures.csv')

In [10]:
textfeature_df

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,0.103338,0.178215,0.880074,163.583435,0.003195,0.000966,0.005456,3.649114,37.56646,339.463165,...,0.010984,0.009696,0.009947,15.474105,19.077137,22.653144,20.953209,18.644379,15.410624,13.985384


In [11]:
feature = textfeature_df.iloc[:, :].values
feature

array([[ 1.03337660e-01,  1.78214856e-01,  8.80073667e-01,
         1.63583435e+02,  3.19488818e-03,  9.65675309e-04,
         5.45620697e-03,  3.64911366e+00,  3.75664597e+01,
         3.39463165e+02,  1.05876266e+02, -1.10000000e-01,
         6.72183736e-02,  2.25411145e-01,  4.86675035e-02,
        -1.44570587e+02,  4.78772888e+01, -9.86902714e+00,
        -1.52537565e+01, -7.70267439e+00, -8.23727608e+00,
        -1.04991837e+01,  6.62301111e+00, -1.56702113e+00,
         2.57743573e+00,  5.54723549e+00, -7.32690573e-01,
         2.79931605e-01,  4.51059532e+01,  2.50011387e+01,
         2.05473213e+01,  1.63323975e+01,  1.05645828e+01,
         1.01263266e+01,  1.34625568e+01,  1.03009682e+01,
         9.95821571e+00,  6.37480450e+00,  8.59253025e+00,
         8.05870152e+00,  9.22345734e+00, -2.15887380e+00,
         9.97951355e+01,  2.42222900e+01,  1.25691795e+01,
         3.53082581e+01,  1.35889244e+01,  2.83860855e+01,
         4.41883049e+01,  1.48488331e+01,  2.99393997e+0

In [12]:
df_features = pd.read_csv('features.csv')

In [13]:
scaler = StandardScaler()
X = df_features.iloc[:, 1:].values
X = scaler.fit_transform(X)
X.shape

(15735, 201)

In [14]:
feature = scaler.transform(feature)

In [15]:
feature = np.expand_dims(feature, axis=2)
feature.shape

(1, 201, 1)

In [16]:
pred_test = model.predict(feature)

2022-05-06 02:00:58.603841: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-06 02:00:58.604238: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-05-06 02:00:58.654815: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [17]:
pred_test

array([[1.7243538e-19, 1.8075547e-16, 5.1680848e-14, 1.0000000e+00]],
      dtype=float32)

In [18]:
pred_test.argmax(axis=1)

array([3])

In [19]:
pred_test

array([[1.7243538e-19, 1.8075547e-16, 5.1680848e-14, 1.0000000e+00]],
      dtype=float32)

In [20]:
Y = df_features['labels'].values

In [21]:
encoder = OneHotEncoder()
print(np.array(Y).reshape(-1, 1))
Y = encoder.fit_transform(np.array(Y).reshape(-1, 1)).toarray()

[['taunt']
 ['taunt']
 ['taunt']
 ...
 ['upset']
 ['upset']
 ['upset']]


In [41]:
test_label_1 = np.array([[1.0, 0.0, 0.0, 0.0]],dtype=np.float32)
test_label_2 = np.array([[0.0, 1.0, 0.0, 0.0]],dtype=np.float32)
test_label_3 = np.array([[0.0, 0.0, 1.0, 0.0]],dtype=np.float32)
test_label_4 = np.array([[0.0, 0.0, 0.0, 1.0]],dtype=np.float32)

In [42]:
test_label_1

array([[1., 0., 0., 0.]], dtype=float32)

In [43]:
print(encoder.inverse_transform(test_label_1))
print(encoder.inverse_transform(test_label_2))
print(encoder.inverse_transform(test_label_3))
print(encoder.inverse_transform(test_label_4))

[['angry']]
[['calm']]
[['taunt']]
[['upset']]


In [44]:
Y_pred = encoder.inverse_transform(pred_test)
Y_pred

array([['upset']], dtype=object)

In [45]:
# loading json and model architecture 
json_file = open('model_json_aug.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("models/Emotion_Model.h5")
print("Loaded model from disk")

Loaded model from disk


In [47]:
print(type(Y_pred))
print(Y_pred.item(0))
print(type(Y_pred.item(0)))

<class 'numpy.ndarray'>
upset
<class 'str'>


In [5]:
# Keras optimiser
loaded_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 201, 256)          1536      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 101, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 101, 256)          327936    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 51, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 51, 128)           163968    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 26, 128)           0         
_________________________________________________________________
dropout (Dropout)            (None, 26, 128)           0

In [6]:
import pyaudio
import wave
from datetime import datetime
import speech_recognition as sr

FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 10
WAVE_OUTPUT_FILENAME = ""

pa = pyaudio.PyAudio()
r = sr.Recognizer()

In [7]:
stream = pa.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True, frames_per_buffer=CHUNK, input_device_index=0)
stream.start_stream()
print("開始錄音......")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)
print("錄音結束!")

stream.stop_stream()
stream.close()
pa.terminate()

WAVE_OUTPUT_FILENAME = "./record/"+datetime.now().strftime("%Y-%m-%d-%H-%M-%S")+".wav"

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(pa.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

開始錄音......
錄音結束!


In [8]:

with sr.AudioFile(WAVE_OUTPUT_FILENAME) as source:
    audio = r.record(source)

In [16]:
output_df = pd.DataFrame(columns=['features'])

output_df.features = get_features(WAVE_OUTPUT_FILENAME)

In [17]:
output_feature_df = pd.DataFrame(output_df['features'].values.tolist()).T


In [19]:
feature = output_feature_df.iloc[:, :].values
feature = scaler.transform(feature)
feature = np.expand_dims(feature, axis=2)
pred_test = loaded_model.predict(feature)

2022-04-29 00:52:41.035676: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-29 00:52:41.036198: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-29 00:52:41.073640: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [23]:
Y_pred = encoder.inverse_transform(pred_test)
Y_pred

array([['upset']], dtype=object)